# Magnetfeld stationärer Ströme

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook wird die Berechnung des Magnetfelds stationärer Ströme am Beispiel von Kreisströmen demonstriert. Im ersten Teil wird ein einzelner Kreisstrom betrachtet. Dazu wird eine numerische Integration des Gesetzes von Biot-Savart durchgeführt,
das in skalierten Variablen durch

$$\boldsymbol{B}(\boldsymbol{r}) = \int \text{d} \boldsymbol{r}' \; \frac{\boldsymbol{I}(\boldsymbol{r}') \times (\boldsymbol{r}-\boldsymbol{r}')}{|\boldsymbol{r}-\boldsymbol{r}'|^3}$$

gegeben ist. $\boldsymbol{r}'$ ist dabei ein Vektor, der vom Ursprung auf einen Punkt auf dem stromdurchflossenen Kreis zeigt. Alternativ werden wir die analytische Lösung auswerten, die die Berechnung elliptischer Integrale erfordert. In Zylinderkoordinaten ist das Magnetfeld durch

$$B_\rho = \frac{2}{\sqrt{(\rho+1)^2+z^2}} \left( \frac{1+\rho^2+z^2}{(1-\rho)^2+z^2} E(k^2) - K(k^2) \right) \frac{z}{\rho} \\
B_z = \frac{2}{\sqrt{(\rho+1)^2+z^2}} \left( \frac{1-\rho^2-z^2}{(1-\rho)^2+z^2} E(k^2) - K(k^2) \right)$$

gegeben, wobei $K$ und $E$ die vollständigen elliptischen Integrale erster und zweiter Art sind und

$$
k^2 = \frac{4\rho}{(1+\rho)^2+z^2}
$$

ist.

Im zweiten Teil des Jupyter-Notebooks wird dann das Magnetfeld zweier zueinander senkrecht stehender Kreisströme betrachtet. Während bei einem einzelnen Kreisstrom die Feldlinien geschlossen sind, wird dies bei dem System zweier Kreisströme im Allgemeinen nicht der Fall sein, wie wir sowohl anhand eines Feldlinienbilds als auch anhand eines Poincaré-Plots zeigen werden.

## Magnetfeld eines einzelnen Kreisstroms

### Importanweisungen

In [ ]:
from functools import partial
from math import cos, pi, sin, sqrt
import numpy as np
import numpy.linalg as LA
from scipy import integrate, special
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

### Beitrag eines infinesimalen Leiterabschnitts zum Magnetfeld

Die Funktion `dB` berechnet den Integranden im Gesetz von Biot-Savart. Dabei ist `r` der Ort, an dem das Magnetfeld zu berechnen ist und `phi` parametrisiert den Ort `r_prime` auf dem Einheitskreis. `e_phi` ist der Tangentialvektor an den Kreis und `dB` ist der sich ergebende Beitrag zum Magnetfeld.

In [ ]:
def dB(phi, r):
    r_prime = np.array([cos(phi), sin(phi), 0])
    e_phi = np.array([-sin(phi), cos(phi), 0])
    dB = np.cross(e_phi, r-r_prime) / LA.norm(r-r_prime)**3
    return dB

### Numerische Berechnung des Magnetfelds an einem vorgegebenen Ort

In `dB_partial` wird der Ort fixiert, so dass nur das erste Argument von `dB`, also `phi` als Variable von der Integrationsroutine übergeben werden muss. Um eine vektorwertige Funktion integrieren zu können, wird hier die Funktion `quad_vec` aus dem SciPy-Paket verwendet.

In [ ]:
def b_numerical(r):
    dB_partial = partial(dB, r=r)
    b, err = integrate.quad_vec(dB_partial, -pi, pi)
    return b

### Analytische Berechnung des Magnetfelds an einem vorgegebenen Ort

Das Integral, das in `b_numerical` numerisch gelöst wird, kann auch analytisch berechnet werden, wobei man das in der Einleitung angegebene Ergebnis in Zylinderkoordinaten erhält. Da wir die analytische Lösung auch bei der Behandlung des Problems zweier Kreisströme verwenden wollen, können der Kreisursprung `r_0` und der Normalenvektor `normal` auf der Kreisfläche vorgegeben werden, wobei der Normalenvektor im Bezug auf die Stromrichtung durch die Rechte-Hand-Regel festgelegt ist. Werden `r_0` und `normal` nicht angegeben, so wird der Kreis am Ursprung zentriert und die Normalenrichtung ist durch die $z$-Richtung gegeben.
Die Zylinderkoordinate $z$ ergibt sich durch die Projektion des Vektors vom Kreismittelpunkt $\boldsymbol{r}_0$ zum Punkt $\boldsymbol{r}$ auf die Normalenrichtung und `rhovec` beschreibt die Projektion von $\boldsymbol{r}-\boldsymbol{r}_0$ auf die Ebene senkrecht zum Normalenvektor. Die elliptischen Integrale E und K werden mit Hilfe der Funktionen `special.ellipe` bzw. `special.ellipk` aus dem SciPy-Paket berechnet. Am Ende werden die Komponenten mit Hilfe der Basisvektoren `normal` und `rhovec/rho` zum Magnetfeldvektor zusammengesetzt.

In [ ]:
def b_one_loop(r, r_0=None, normal=None):
    if r_0 is None:
        r_0 = np.array([0, 0, 0])
    if normal is None:
        normal = np.array([0, 0, 1])

    z = (r-r_0) @ normal
    rhovec = r - r_0 - z*normal
    rho = LA.norm(rhovec)
    prefactor = 2/sqrt((rho+1)**2+z**2)
    k_squared = 4*rho/((1+rho)**2+z**2)
    ellip_e = special.ellipe(k_squared)
    ellip_k = special.ellipk(k_squared)

    e_factor = (1+rho**2+z**2) / ((1-rho)**2+z**2)
    b_rho = prefactor*(e_factor*ellip_e - ellip_k) * z/rho

    e_factor = (1-rho**2-z**2) / ((1-rho)**2+z**2)
    b_z = prefactor*(e_factor*ellip_e + ellip_k)
    return b_z*normal + b_rho*rhovec/rho

### Implementierung der Differentialgleichung für die Feldlinien

Die Magnetfeldlinien sind dadurch definiert, dass an jedem Punkt der Feldlinie die Tangentialrichtung durch das Magnetfeld gegeben ist. Damit die Feldlinie mit konstanter Geschwindigkeit durchlaufen wird, wird das magnetische Feld normiert. Um eine Division durch null im Falle eines verschwindenden Magnetfelds zu vermeiden, wird eine sehr kleine Zahl im Nenner addiert.

In [ ]:
def dr_dt(t, r, b_field, r_start, eps):
    b = b_field(r)
    return b / (LA.norm(b)+1.e-6)

Die Berechnung wird beendet, wenn die durch `line_closed` gegebene Bedingung für das Schließen der Feldlinie erfüllt ist.

In [ ]:
def line_closed(t, r, b_field, r_start, eps):
    return LA.norm(r-r_start) - eps
line_closed.terminal = True
line_closed.direction = -1

### Berechnung einer einzelnen Feldlinie

Die Berechnung einer einzelnen Feldlinie erfolgt als Lösung eines Anfangswertproblems mit Hilfe der SciPy-Funktion `integrate.solve_ivp`. Wenn das Schließen der Feldlinie festgestellt wurde, werden nur die Daten für einen entsprechend eingeschänkten Wertebereich für die Zeiten `t_values` übergeben. Mit Hilfe des Arguments `b_field` wird die Funktion festgelegt, die zur Berechnung des Magnetfelds verwendet wird. Dies kann entweder `b_numerical` oder `b_one_loop` sein.

In [ ]:
def one_field_line(b_field, t_end, n_max, r_start, eps):
    t_values = np.linspace(0, t_end, n_max)
    solution = integrate.solve_ivp(
        dr_dt, (0, t_end), r_start,
        args=(b_field, r_start, eps),
        events=line_closed, dense_output=True,
        atol=1e-10, rtol=1e-10, t_eval=t_values)
    if solution.t_events[0].size > 0:
        t_end = solution.t_events[0][0]
        t_values = np.linspace(0, t_end, n_max)
    return solution.sol(t_values)

### Berechnung aller Feldlinien

Es werden ein Satz an Feldlinien berechnet, die in verschiedenen Abständen vom Kreismittelpunkt auf der $x$-Achse starten. Im Argument `b_field` wird die Funktion übergeben, die zur Berechnung des Magnetfelds verwendet werden soll. Die Feldlinien werden als Liste von Listen der Punkte je Feldlinie übergeben.

In [ ]:
def all_field_lines(b_field, t_end, n_max, eps,
                    r_start_max, n_start_max):
    field_lines = []
    for r_start in np.linspace((r_start_max, 0, 0),
                               (1, 0, 0), n_start_max,
                               endpoint=False):
        field_line = one_field_line(b_field, t_end, n_max,
                                    r_start, eps)
        field_lines.append(field_line)
    return field_lines

### Implementierung der Bedienelemente und graphische Darstellung der Feldlinien

Mit Hilfe der Bedienelemente lassen sich die folgenden Parameter einstellen:
- `method_kw`: Funktion zur Berechnung des Magnetfelds
- `t_end`: maximale Länge der Feldlinien
- `n_max`: Anzahl der Punkte auf der Feldlinie
- `r_start_max`: maximaler Abstand vom Kreismittelpunkt für die Startpunkte auf der $x$-Achse
- `n_start_max`: Anzahl der verschiedenen Startpunkte
- `eps`: Fehlertoleranz der Bedingung für das Schließen einer Feldlinie

Die Funktion `all_field_lines` berechnet die Feldlinien. Wird das Magnetfeld über die analytischen Ausdrücke berechnet, so erfolgt die Darstellung in schwarz, bei einer numerischen Berechnung dagegen in blau. Im zweiten Fall kann die Rechnung einige Zeit in Anspruch nehmen. Die Feldlinien werden nur für Startwerte auf der positiven $x$-Achse berechent, jedoch symmetrisch auf beiden Seiten des Ursprungs dargestellt.

In [ ]:
widget_dict = {"method_kw":
               widgets.RadioButtons(
                   options=["analytisch", "numerisch"],
                   value="analytisch",
                   description="Berechnung des Magnetfelds"),
               "t_end":
               widgets.FloatSlider(
                   value=50, min=10, max=50, step=10,
                   description=r"$t_\text{end}$"),
               "n_max":
               widgets.IntSlider(
                   value=200, min=100, max=1000, step=100,
                   description=r"$n_\text{max}$"),
               "r_start_max":
               widgets.FloatSlider(
                   value=2, min=1.5, max=10, step=0.5,
                   description=r"$r_\text{start max}$"),
               "n_start_max":
               widgets.IntSlider(
                   value=6, min=2, max=10, step=1,
                   description=r"$n_\text{start max}$"),
               "eps":
               widgets.FloatSlider(
                   value=0.01, min=0.002, max=0.01,
                   step=0.002, readout_format=".3f",
                   description=r"$\varepsilon$")
               }

@interact(**widget_dict)
def plot_one_loop_result(method_kw, t_end, n_max,
                         r_start_max, n_start_max, eps):
    if method_kw == "analytisch":
        method = b_one_loop
        linecolor = "black"
    else:
        method = b_numerical
        linecolor = "blue"
    field_lines = all_field_lines(method,
                                  t_end, n_max, eps,
                                  r_start_max, n_start_max)

    fig, ax = plt.subplots()
    for field_line in field_lines:
        ax.plot(field_line[0], field_line[2],
                color=linecolor)
        ax.plot(-field_line[0], field_line[2],
                color=linecolor)
    ax.set_aspect("equal")
    ax.set_xlabel("$x$")
    ax.set_ylabel("$y$")

## Magnetfeld zweier orthogonaler Kreisströme

Betrachtet wird das Magnetfeld, das durch zwei orthogonale, kreisströmige Leiterschleifen mit der gleichen Stromstärke erzeugt wird. Dabei liegt der Kreismittelpunkt der einen Leiterschleife auf der anderen Leiterschleife.

In [ ]:
def b_two_loops(r):
    b_1 = b_one_loop(r)
    b_2 = b_one_loop(r,
                     r_0=np.array([-1, 0, 0]),
                     normal=np.array([0, -1, 0]))
    return b_1 + b_2

### Implementierung der Bedienelemente und graphische Darstellung der Feldlinie

Mit Hilfe der Schieberegler können die folgenden Parameter eingestellt werden:
- `t_end`: maximale Länge der Feldlinie
- `n_max`: Anzahl der Punkte auf der Feldlinie
- `x_start`: $x$-Koordinate des Startpunkts der Feldlinie
- `y_start`: $y$-Koordinate des Startpunkts der Feldlinie
- `z_start`: $z$-Koordinate des Startpunkts der Feldlinie
- `phi`: Azimutwinkel, wird zur Rotation der Feldlinie in `plot_inner` verwendet
- `theta`: Polarwinkel, wird zur Rotation der Feldlinie in `plot_inner` verwendet

Die Berechnung der Feldlinie erfolgt mittels der zuvor verwendeten Funktion `one_field_line` unter Verwendung der Funktion `b_two_loops` für das Magnetfeld zweier Leiterschleifen. Die Feldlinie wird in einer dreidimensionalen Darstellung gezeigt, die durch Einstellung der Winkel `phi` und `theta` gedreht werden kann. Die beiden Leiterschleifen werden durch rote Kreise dargestellt.

In [ ]:
widget_dict = {"t_end":
               widgets.FloatSlider(
                   value=50, min=10, max=500, step=10,
                   description=r"$t_\text{end}$"),
               "n_max":
               widgets.IntSlider(
                   value=200, min=100, max=1000, step=100,
                   description=r"$n_\text{max}$"),
               "x_start":
               widgets.FloatSlider(
                   value=0.5, min=-2, max=2, step=0.05,
                   description=r"$x_\text{start}$"),
               "y_start":
               widgets.FloatSlider(
                   value=0.2, min=-2, max=2, step=0.05,
                   description=r"$y_\text{start}$"),
               "z_start":
               widgets.FloatSlider(
                   value=0, min=-2, max=2, step=0.05,
                   description=r"$z_\text{start}$"),
               "phi":
               widgets.FloatSlider(
                   value=-120, min=-180, max=180, step=5,
                   description=r"$\phi$"),
               "theta":
               widgets.FloatSlider(
                   value=15, min=-90, max=90, step=5,
                   description=r"$\theta$"),
               }

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(**widget_dict)
def plot_two_loop_result(t_end, n_max,
                         x_start, y_start, z_start):
    r_start = np.array([x_start, y_start, z_start])
    eps = 0
    field_line = one_field_line(b_two_loops, t_end, n_max,
                                r_start, eps)

    @interact(**widget_dict)
    def plot_inner(theta, phi):
        ax = plt.axes(projection="3d")
        ax.view_init(theta, phi)
        ax.plot(*field_line, color="black", lw=1)
        phi = np.linspace(0, 2*pi, 200)
        x = np.cos(phi)
        y = np.sin(phi)
        z = np.zeros_like(phi)
        ax.plot(x, y, color="red")
        ax.plot(x-1, z, y, color="red")
        ax.set_xlabel("$x$")
        ax.set_ylabel("$y$")
        ax.set_zlabel("$z$")
        ax.set_aspect("equal", adjustable="box")
        plt.show()

### Bedingungen für Poincaré-Schnitt

Zur Darstellung des Poincaré-Schnitts werden die Durchstoßpunkte durch die $x$-$y$-Ebene detektiert, wobei beide Durchstoßrichtungen zugelassen sind. In der graphischen Darstellung werden später die unterschiedlichen Durchstoßrichtungen farblich unterschieden.

In [ ]:
def z_equal_zero_pos(t, r, *args):
    b_field, r_start, eps = args
    return r[2]
z_equal_zero_pos.terminal = True
z_equal_zero_pos.direction = 1

In [ ]:
def z_equal_zero_neg(t, r, *args):
    b_field, r_start, eps = args
    return r[2]
z_equal_zero_neg.terminal = True
z_equal_zero_neg.direction = -1

### Berechnung des Poincaré-Plots

Zur Berechnung des Poincaré-Plots wird die Feldlinie stückweise numerisch integriert bis ein Durchstoßpunkt durch die $x$-$y$-Ebene unabhängig von der Durchstoßrichtung detektiert wurde. Das NumPy-Array `points` enthält am Ende `n_out` Koordinatentupel, wobei sich die Durchstoßrichtungen abwechseln. Da die Rechnung typischerweise etwas Zeit erfordert, wird in regelmäßigen Abständen die Anzahl der bereits berechneten Poincaré-Punkte ausgegeben.

In [ ]:
def poincare_points(dt, b_field, r_0, n_out):
    points = np.zeros((2, n_out))
    t_start = 0
    t_end = dt
    events = (z_equal_zero_pos, z_equal_zero_neg)
    for cnt in range(n_out):
        found = False
        while not found:
            solution = integrate.solve_ivp(
                dr_dt, (t_start, t_end), r_0,
                args=(b_field, r_0, 0.),
                events=events[cnt % 2],
                dense_output=True, atol=1e-10, rtol=1e-10)
            if np.size(solution.t_events) > 0:
                points[:, cnt] = solution.y_events[0][0, :2]
                found = True
            t_start = solution.t[-1]
            t_end = t_start + dt
            r_0 = solution.y[:, -1]
        if 10*cnt % n_out == 0:
            print(cnt)
    return points

### Implementierung der Bedienelemente und graphische Darstellung des Poincaré-Plots

Mit Hilfe der Schieberegler lassen sich die folgenden Parameter einstellen:
- `dt`: Länge eines Integrationsintervalls
- `n_out`: Anzahl der Durchstoßpunkte
- `x_start`: $x$-Koordinate des Startpunkts der Feldlinie
- `y_start`: $y$-Koordinate des Startpunkts der Feldlinie
- `z_start`: $z$-Koordinate des Startpunkts der Feldlinie

Es werden die Durchstoßpunkte einer Feldlinie durch die $x$-$y$-Ebene bestimmt und als Poincaré-Plot dargestellt. Dabei werden entgegengesetzte Durchstoßrichtungen blau bzw. grün dargestellt.

In [ ]:
widget_dict = {"dt":
               widgets.FloatSlider(
                   value=20, min=1, max=100, step=1,
                   description=r"$\Delta t$"),
               "n_out":
               widgets.IntSlider(
                   value=1000, min=1000, max=20000,
                   step=1000,
                   description=r"$n_\text{out}$"),
               "x_start":
               widgets.FloatSlider(
                   value=0.5, min=-2, max=2, step=0.05,
                   description=r"$x_\text{start}$"),
               "y_start":
               widgets.FloatSlider(
                   value=0.2, min=-2, max=2, step=0.05,
                   description=r"$y_\text{start}$"),
               "z_start":
               widgets.FloatSlider(
                   value=0, min=-2, max=2, step=0.05,
                   description=r"$z_\text{start}$")
               }

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(**widget_dict)
def plot_poincare(dt, n_out, x_start, y_start, z_start):
    r_0 = np.array([x_start, y_start, z_start])
    points = poincare_points(dt, b_two_loops, r_0, n_out)

    fig, ax = plt.subplots()
    ax.plot(points[0, ::2], points[1, ::2], color="green",
            linestyle="None", marker="o", markersize=1)
    ax.plot(points[0, 1::2], points[1, 1::2], color="blue",
            linestyle="None", marker="o", markersize=1)
    ax.set_xlabel("$x$")
    ax.set_ylabel("$y$")
    ax.set_xlim((-9, 8))
    ax.set_ylim((-5, 5))